# Using MVR to predict gz from well log data

Put together by Thomas Martin, thomasmartin@mines.edu, all errors are mine

In [ ]:
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in /usr/local/lib/python3.6/dist-packages (0.0)


In [ ]:
# If you have installation questions, please reach out
import pandas as pd # data storage
import numpy as np  # math and stuff

import sklearn  
import datetime

from sklearn import linear_model
import statsmodels.api as sm

from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import median_absolute_error, max_error, mean_squared_error
from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt # plotting utility

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('drive/My Drive/1_lewis_research/core_to_wl_merge/Merged_dataset_inner_imputed_12_21_2020.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'LiveTime2','ScanTime2', 'LiveTime1','ScanTime1',
              'ref_num', 'API', 'well_name', 'sample_num', 'sample' ], axis=1)

In [ ]:
df = df[df.gz_pchip_interp >= 0]

In [ ]:
df

,depth_ft,USGS_ID,CAL,GR,DT,SP,DENS,PE,RESD,PHIN,PHID,PE_smooth,GR_smooth,gz_linear_interp,gz_pchip_interp,perm_air_md,perm_klink_md,He_por,horz_perm_md,effective_por,por_percent,stress_swanson_perm_md,Na,Na Err,Mg,Mg Err,Al,Al Err,Si,Si Err,P,P Err,S,S Err,K,K Err,Ca,Ca Err,Ti,Ti Err,Cr,Cr Err,Mn,Mn Err,Fe,Fe Err,Co,Co Err,Ni,Ni Err,Cu,Cu Err,Zn,Zn Err,Ba,Ba Err
0,7741.0,F028,0.630495,0.187403,0.054025,0.582081,0.829176,0.349137,0.982069,0.001692,0.001692,0.366294,0.183350,4.0313,4.0320,NaN,NaN,NaN,NaN,NaN,14.5,5.55,1.2770,0.0263,0.6987,0.0116,2.8101,0.0112,33.9921,0.0332,0.0826,0.0021,0.1116,0.0011,0.9947,0.0045,4.5640,0.0073,0.1641,0.0013,0.0085,0.0001,0.0186,0.0008,1.4531,0.0029,0.0011,0.0,0.0044,0.0001,0.0,0.0008,0.0023,0.0008,0.0663,0.0049
1,7741.1,F028,0.629618,0.186340,0.054025,0.583138,0.828532,0.346560,0.970108,0.001674,0.001674,0.364550,0.185719,3.9834,3.9834,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7741.2,F028,0.628741,0.185373,0.054025,0.584257,0.827725,0.349993,0.958103,0.001662,0.001662,0.362606,0.188207,3.9837,3.9836,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7741.3,F028,0.627865,0.184035,0.056538,0.585297,0.827017,0.345159,0.946262,0.001721,0.001721,0.360835,0.190811,3.9840,3.9840,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7741.4,F028,0.626988,0.182885,0.060121,0.586086,0.826253,0.345030,0.934387,0.001766,0.001766,0.359328,0.193543,3.9843,3.9843,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12025,13378.0,PDRMT,0.075490,0.638499,0.520073,0.445824,0.502957,0.638966,0.096127,0.509441,0.509441,0.578975,0.598506,4.3717,4.3658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5632,0.0196,0.5084,0.0102,1.8027,0.0094,35.4358,0.0311,0.1020,0.0024,0.2357,0.0015,0.5846,0.0034,4.3886,0.0066,0.1207,0.0012,0.0114,0.0002,0.0153,0.0007,1.0049,0.0024,0.0008,0.0,0.0049,0.0001,0.0,0.0007,0.0034,0.0008,0.1478,0.0056
12026,13378.1,PDRMT,0.075563,0.640781,0.529719,0.448875,0.517158,0.630883,0.085097,0.494742,0.494742,0.579943,0.603776,6.6922,6.7092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12027,13378.2,PDRMT,0.075637,0.641924,0.538041,0.452058,0.531493,0.621742,0.074177,0.479986,0.479986,0.580347,0.608223,6.7190,6.7313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12028,13378.3,PDRMT,0.075710,0.642706,0.540492,0.455364,0.546129,0.612743,0.063364,0.465287,0.465287,0.580152,0.611830,4.3369,4.3273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Loading in Dataset

In [ ]:
df.columns.values

array(['depth_ft', 'USGS_ID', 'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
       'RESD', 'PHIN', 'PHID', 'PE_smooth', 'GR_smooth',
       'gz_linear_interp', 'gz_pchip_interp', 'perm_air_md',
       'perm_klink_md', 'He_por', 'horz_perm_md', 'effective_por',
       'por_percent', 'stress_swanson_perm_md', 'Na', 'Na Err', 'Mg',
       'Mg Err', 'Al', 'Al Err', 'Si', 'Si Err', 'P', 'P Err', 'S',
       'S Err', 'K', 'K Err', 'Ca', 'Ca Err', 'Ti', 'Ti Err', 'Cr',
       'Cr Err', 'Mn', 'Mn Err', 'Fe', 'Fe Err', 'Co', 'Co Err', 'Ni',
       'Ni Err', 'Cu', 'Cu Err', 'Zn', 'Zn Err', 'Ba', 'Ba Err'],
      dtype=object)

In [ ]:
dataset = df[[
  'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
  'RESD', 'PHIN', 'PHID', 'GR_smooth', 'PE_smooth', 'gz_pchip_interp'
]]

In the next code block, we will remove the rows without data, and change string NaN's to np.nans

In [ ]:
dataset.replace('NaN',np.nan, regex=True, inplace=True)# 
# dataset = dataset.dropna()
np.shape(dataset)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


(12030, 12)

In [ ]:
dataset.head(3)

,CAL,GR,DT,SP,DENS,PE,RESD,PHIN,PHID,GR_smooth,PE_smooth,gz_pchip_interp
0,0.630495,0.187403,0.054025,0.582081,0.829176,0.349137,0.982069,0.001692,0.001692,0.183350,0.366294,4.0320
1,0.629618,0.186340,0.054025,0.583138,0.828532,0.346560,0.970108,0.001674,0.001674,0.185719,0.364550,3.9834
2,0.628741,0.185373,0.054025,0.584257,0.827725,0.349993,0.958103,0.001662,0.001662,0.188207,0.362606,3.9836


In [ ]:
X = dataset[[ 'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
        'RESD', 'PHIN', 'PHID', 'GR_smooth', 'PE_smooth']]

Y = dataset[['gz_pchip_interp']]

Y_array = np.array(Y.values)

## Starting to set up the ML model params

In [ ]:
seed = 7 # random seed is only used if you want to compare exact answers with friends 
test_size = 0.25 # how much data you want to withold, .15 - 0.3 is a good starting point

X_train, X_test, y_train, y_test = train_test_split(X.values, Y_array, test_size=test_size)

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

Intercept: 
 [2.76174207]
Coefficients: 
 [[ 0.83134111 -7.15738641 -0.26064256 -0.24421424  0.21625982 -0.54209888
   0.03452909  0.26900565  0.26900565 11.63011994  1.51691665]]


In [ ]:
preds = regr.predict(X_test)

In [ ]:
rmse5 = mean_squared_error(y_test, preds, squared=False)
print("Mean Squared Error: %f" % (rmse5))
max5 = max_error(y_test, preds)
print("Max Error: %f" % (max5))

MAE2 = median_absolute_error(y_test, preds)
print("Median Abs Error: %f" % (MAE2))

Mean Squared Error: 1.371090
Max Error: 5.321322
Median Abs Error: 0.878051


In [ ]:
x = datetime.datetime.now()

d = {'target': [Y.columns.values],
     'MSE': [rmse5],
     'MAE': [MAE2],
     'MaxError': [max5], 
     'day': [x.day], 
     'month':[x.month], 
     'year':[x.year],
     'model':['MVR'],
     'version':[sklearn.__version__]}

results = pd.DataFrame(data=d)

results.to_csv('drive/My Drive/1_lewis_research/analysis/experiments/mvr/mvr_results/grainsize_mvr.csv')

results

,target,MSE,MAE,MaxError,day,month,year,model,version
0,[gz_pchip_interp],1.37109,0.878051,5.321322,15,2,2021,MVR,0.22.2.post1


In [ ]:
from sklearn.metrics import explained_variance_score

In [ ]:
explained_variance_score(y_test, preds, multioutput='uniform_average')

0.3850700308174788